In [ ]:
# Reading the dataset:
import pandas as pd

dataset = pd.read_csv("../data/raw/filtered.tsv", delimiter='\t')
dataset = dataset.set_index(dataset.columns[0])
dataset.index.name = "Index"

In [ ]:
dataset.head()